## Experiments with Nutpie and PYMC built in sampler

In another project I was seeing substantial seed ups (30x) in sampling with nutpie vs the default sampler for a model that fit negative binomial to about 200k samples.  The purpose of this notebook was to produce a minimal example to demonstrate the speed up. 

However, in this notebook after upgrading to recent PYMC, the sampler lets you know real time what the different chains are doing and I see what was happening is that some chains were getting stuck. This didn't seem to happen with the nutpie sampler, but I have not spent time checking that this is not just due to small sample size.   After changing the priors sampling improved for pymc but even so nutpie is a bit faster (only 50% not 30x though!)

In [1]:
import numpy as np
import pymc as pm
import arviz as az
import time

def simulate_data(mu, alpha, num):
    neg_bin = pm.NegativeBinomial.dist(mu=mu, alpha=alpha)
    return pm.draw(neg_bin, num)

test1 = simulate_data(2.7, 0.6, 10000)   
np.mean(test1),np.std(test1)

(2.746, 3.978854609055224)

In [2]:
pm.__version__

'5.21.0'

In [3]:
with pm.Model() as nb_model:
    data = pm.Data('data', test1)
    mu = pm.TruncatedNormal('mu', mu = 2, sigma=5, lower = 0.0)
    alpha = pm.Gamma('alpha', alpha =5, beta =.5 )
    counts = pm.NegativeBinomial('counts',mu=mu, alpha=alpha, shape = data.shape, observed = data)

In [4]:
start = time.perf_counter()


with nb_model:
    trace = pm.sample(1000, tune=1000)  

end = time.perf_counter()

print(f"Elapsed time: {end - start:.6f} seconds")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, alpha]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 36 seconds.


Elapsed time: 70.332254 seconds


In [4]:
test2 = simulate_data(2.7, 0.6, 200000) 
with nb_model:
    pm.set_data({'data': test2})

With default sampler, the chains tend to get stuck (sometimes) and take a long time to finish.

In [5]:
with nb_model:
    start = time.perf_counter()
    trace = pm.sample(1000, tune=1000) 
    end = time.perf_counter()
    print(f"Elapsed time: {end - start:.6f} seconds")

az.summary(trace)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, alpha]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 211 seconds.


Elapsed time: 248.767545 seconds


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,2.708,0.009,2.691,2.724,0.0,0.0,3807.0,2754.0,1.0
alpha,0.601,0.003,0.596,0.606,0.0,0.0,3917.0,2917.0,1.0


Nutpie (so far in my experiments) has not exhibited this behavior

In [6]:
with nb_model:
    start = time.perf_counter()
    trace = pm.sample(1000, tune=1000, nuts_sampler="nutpie")
    end = time.perf_counter()
    print(f"Elapsed time: {end - start:.6f} seconds")


c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pymc\pytensorf.py:952: FutureWarning: compile_pymc was renamed to compile. Old name will be removed in a future release of PyMC
  warnings.warn(
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pymc\pytensorf.py:952: FutureWarning: compile_pymc was renamed to compile. Old name will be removed in a future release of PyMC
  warnings.warn(


Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,1.20,1
,2000,0,1.23,3
,2000,0,1.20,3
,2000,0,1.21,1


Elapsed time: 118.609474 seconds


Only a slide speed up here.   

## Now lets try zero truncated version!

In [4]:
import pytensor.tensor as pt

In [5]:
def logp_ztnb(value, mu, alpha):
    return pm.logp(pm.NegativeBinomial.dist(mu=mu, alpha=alpha),value) - pt.log1mexp(pm.logp(pm.NegativeBinomial.dist(mu=mu, alpha=alpha),0))

def rng_ztnb(mu,alpha, rng = None, size = None):
     p = alpha / (mu + alpha)
     n = alpha
     samples = rng.negative_binomial(n, p, size= size)
     while np.any(samples == 0):
         idx = np.where(samples == 0)
         samples[idx] = rng.negative_binomial(n, p, size= len(idx[0]))
     return samples

In [6]:
rng_ztnb(2.7, 0.6, np.random, 10)

array([12,  1,  4,  2,  2,  2,  1,  2,  1, 10])

In [33]:
logp_ztnb(pt.as_tensor_variable(3), 2.7, 0.6).eval()

array(-2.0563169)

In [7]:
def simulate_data_zt(mu, alpha, num):
    return rng_ztnb(mu, alpha, np.random, num)

test = simulate_data_zt(2.7, 0.6, 200000)
np.mean(test),np.std(test)

(4.21829, 4.11426779341112)

In [8]:
min(test)

1

In [52]:
with pm.Model() as ztnb_model:
    data = pm.Data('data', test)
    mu = pm.TruncatedNormal('mu', mu = 2, sigma=5, lower = 0.0)
    alpha = pm.Gamma('alpha', alpha =5, beta =.5 )
    counts = pm.CustomDist('counts',mu, alpha, logp = logp_ztnb, random = rng_ztnb, shape = data.shape, observed = data)

In [53]:
start = time.perf_counter()


with ztnb_model:
    trace = pm.sample(1000, tune=1000)  

end = time.perf_counter()

print(f"Elapsed time: {end - start:.6f} seconds")
az.summary(trace)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, alpha]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 171 seconds.


Elapsed time: 172.084338 seconds


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,2.719,0.014,2.695,2.746,0.0,0.0,969.0,1673.0,1.0
alpha,0.604,0.006,0.592,0.615,0.0,0.0,931.0,1553.0,1.0


In [54]:
start = time.perf_counter()


with ztnb_model:
    trace = pm.sample(1000, tune=1000, nuts_sampler="nutpie")  

end = time.perf_counter()

print(f"Elapsed time: {end - start:.6f} seconds")
az.summary(trace)

c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pymc\pytensorf.py:952: FutureWarning: compile_pymc was renamed to compile. Old name will be removed in a future release of PyMC
  warnings.warn(
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pymc\pytensorf.py:952: FutureWarning: compile_pymc was renamed to compile. Old name will be removed in a future release of PyMC
  warnings.warn(


Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,0.84,1
,2000,0,0.77,7
,2000,0,0.82,7
,2000,0,0.80,3


Elapsed time: 175.783781 seconds


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_interval__,1.000,0.005,0.990,1.010,0.000,0.0,825.0,1549.0,1.0
alpha_log__,-0.504,0.011,-0.524,-0.484,0.000,0.0,828.0,1438.0,1.0
mu,2.719,0.015,2.691,2.746,0.001,0.0,825.0,1549.0,1.0
alpha,0.604,0.007,0.592,0.616,0.000,0.0,828.0,1438.0,1.0


In [9]:
test = simulate_data_zt(2.7, 0.6, 200000)  # smaller data for this test

In [10]:

with pm.Model() as ztnb_truncated_model:
    data = pm.Data('data', test)
    mu = pm.TruncatedNormal('mu', mu=2, sigma=5, lower=0.0)
    alpha = pm.Gamma('alpha', alpha=5, beta=0.5)

    # try to use truncated built in.
    counts = pm.Truncated(
        'counts',
        pm.NegativeBinomial.dist(mu=mu, alpha=alpha),
        lower=1,  # truncation at 0 means support from 1 upwards
        observed=data
    )

In [61]:
with ztnb_truncated_model:
    trace = pm.sample(1000, tune=1000)

az.summary(trace)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, alpha]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 197 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,2.713,0.014,2.687,2.739,0.0,0.0,1050.0,1773.0,1.01
alpha,0.606,0.006,0.594,0.618,0.0,0.0,1051.0,1650.0,1.01


Nutpie has some issues doing it this way, warnings and it takes a bit longer .

In [11]:
with ztnb_truncated_model:
    trace = pm.sample(1000, tune=1000, nuts_sampler="nutpie")

az.summary(trace)

c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pytensor\link\numba\dispatch\basic.py:373: UserWarning: Numba will use object mode to run betainc_grad's perform method
  warnings.warn(
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pytensor\link\numba\dispatch\basic.py:373: UserWarning: Numba will use object mode to run betainc_grad's perform method
  warnings.warn(


Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,0.74,3
,2000,0,0.77,7
,2000,0,0.77,3
,2000,0,0.81,3


c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pytensor\scalar\basic.py:2106: RuntimeWarning: divide by zero encountered in divide
  return x / y
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pytensor\scalar\basic.py:2106: RuntimeWarning: invalid value encountered in divide
  return x / y
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pytensor\scalar\basic.py:1964: RuntimeWarning: invalid value encountered in scalar add
  return sum(inputs)
c:\Users\rjljr\miniconda3\envs\nutpie\Lib\site-packages\pytensor\scalar\basic.py:2064: RuntimeWarning: invalid value encountered in scalar

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_interval__,0.989,0.005,0.979,0.999,0.0,0.0,990.0,1463.0,1.01
alpha_log__,-0.527,0.011,-0.546,-0.507,0.0,0.0,1095.0,1556.0,1.01
mu,2.689,0.014,2.661,2.714,0.0,0.0,990.0,1463.0,1.01
alpha,0.591,0.006,0.579,0.602,0.0,0.0,1095.0,1556.0,1.01


In [12]:
import nutpie
nutpie.__version__

'0.14.2'